# Readability Assessment through Learning Techniques

* Readability assessment is a well known problem in natural language processing field. 
* Giving someone the suitable text for his level of comprehension (not so easy and not so hard) could maximize his understanding and enjoyment. 

In this notebook we are trying to assess the readability of a given text regardless of the text topic.

## Corpus
> OneStopEnglish corpus: A new corpus for automatic readability assessment and text simplification  
> Sowmya Vajjala and Ivana Lučić  
> 2018  
> Proceedings of the Thirteenth Workshop on Innovative Use of NLP for Building Educational Applications, pages 297–304. Association for Computational Linguistics.  
> [url](http://aclweb.org/anthology/W18-0535). [bib file](https://aclanthology.coli.uni-saarland.de/papers/W18-0535/w18-0535.bib)

Please cite the above paper if you use this corpus in your research.

[![DOI](https://zenodo.org/badge/128919409.svg)](https://zenodo.org/badge/latestdoi/128919409)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

Now let's dive into our corpus.

In [1]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from ipywidgets import interact
from tabulate import tabulate
from IPython.display import display, HTML

from utils.loading import load_glove_embeddings, load_old_corpus
from utils.one_stop_english import load_corpus, corpus_to_words, corpus_to_pos, detokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional,Conv1D,MaxPooling1D,GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np

corpus = load_corpus()
articles_words, tags = corpus_to_words(corpus)
articles_pos, tags = corpus_to_pos(corpus)
old_articles_pos, old_articles_tags = load_old_corpus()

Reading level|Avg. Num. Words|Std. Dev|Number of Articles
---|---|---|---
Elementary|533.17|103.79|189
Intermediate|676.59|117.15|189
Advanced|820.49|162.52|189



In [2]:
@interact
def show_articles(i=(0,188,1), words=(0,1000,1)):
    data = [
        ["Advanced",detokenize(articles_words[i][:words])], 
        ["Intermediate",detokenize(articles_words[i+2][:words])], 
        ["Elementary",detokenize(articles_words[i+1][:words])]
    ]
    headers = ['Reading Level', 'Example']
    display(HTML(tabulate(data,tablefmt='html', headers=headers)+"<style>th,td {font-size: 20px}</style>"))

interactive(children=(IntSlider(value=94, description='i', max=188), IntSlider(value=500, description='words',…

## Data preprocessing

### 1. Converting words to part of speech tags

In [3]:
print(articles_words[0][:10])
print(articles_pos[0][:10])

['When', 'you', 'see', 'the', 'word', 'Amazon', ',', 'whats', 'the', 'first']
['WRB', 'PRP', 'VB', 'DT', 'NN', 'NN', ',', 'VBZ', 'DT', 'JJ']


### 2. Converting part of speech to sequences

In [4]:
maxlen = 1000 # Cuts off reviews after 1000 words
max_words = 45
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(articles_pos)

In [5]:
sequences = tokenizer.texts_to_sequences(articles_pos)
old_sequences = tokenizer.texts_to_sequences(old_articles_pos)
print(np.shape(sequences))
print(np.shape(old_sequences))

(567,)
(183,)


### 3. Padding

In [6]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
old_data = pad_sequences(old_sequences, maxlen=maxlen, padding='post', truncating='post')
print(np.shape(data))
print(np.shape(old_data))

(567, 1000)
(183, 1000)


In [7]:
tags = np.array(tags)
old_tags = np.array(old_articles_tags)
print(tags.shape)
print(old_tags.shape)

(567,)
(183,)


### 4. Convert sequences to one hot encoding

In [8]:
data = to_categorical(data)
old_data = to_categorical(old_data)
print(data.shape)
print(old_data.shape)

(567, 1000, 45)
(183, 1000, 45)


### 5. Randomize data

In [9]:
np.random.seed(0)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
tags = tags[indices]

old_indices = np.arange(old_data.shape[0])
np.random.shuffle(indices)

old_data = old_data[old_indices]
old_tags = old_tags[old_indices]
print(data.shape)
print(tags.shape)

print(old_data.shape)
print(old_tags.shape)

(567, 1000, 45)
(567,)
(183, 1000, 45)
(183,)


## Structuring the model

In [117]:
model = Sequential()
model.add(Conv1D(7, 10, activation='relu',kernel_regularizer=l1(0.038), input_shape=(1000,45)))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.0003, l2=0.0003)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1))
def soft_acc(y_true, y_pred):
    from tensorflow.python.keras import backend as K
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [ ]:
es = EarlyStopping(monitor='val_soft_acc',patience=100, mode='max')
cp = ModelCheckpoint('model-{epoch:03d}-{soft_acc:03f}-{val_soft_acc:03f}.h5', verbose=0, monitor='val_soft_acc',save_best_only=True, mode='max')  
model.compile(optimizer=RMSprop(),loss='mse',metrics=[soft_acc])
history = model.fit(data, tags,epochs=2000,  batch_size=50,validation_split=0.2,callbacks=[cp], verbose=1)

Train on 453 samples, validate on 114 samples
Epoch 1/2000
453/453 [==============================] - 3s 8ms/step - loss: 7.6696 - soft_acc: 0.3355 - val_loss: 6.5118 - val_soft_acc: 0.3246
Epoch 2/2000
453/453 [==============================] - 0s 295us/step - loss: 5.7842 - soft_acc: 0.3311 - val_loss: 5.1584 - val_soft_acc: 0.3246
Epoch 3/2000
453/453 [==============================] - 0s 304us/step - loss: 4.6190 - soft_acc: 0.3510 - val_loss: 4.1569 - val_soft_acc: 0.3246
Epoch 4/2000
453/453 [==============================] - 0s 289us/step - loss: 3.7293 - soft_acc: 0.3466 - val_loss: 3.3865 - val_soft_acc: 0.3158
Epoch 5/2000
453/453 [==============================] - 0s 287us/step - loss: 3.0297 - soft_acc: 0.3422 - val_loss: 2.7117 - val_soft_acc: 0.3070
Epoch 6/2000
453/453 [==============================] - 0s 296us/step - loss: 2.4334 - soft_acc: 0.3400 - val_loss: 2.2045 - val_soft_acc: 0.3070
Epoch 7/2000
453/453 [==============================] - 0s 282us/step - loss: 1.

453/453 [==============================] - 0s 273us/step - loss: 0.7461 - soft_acc: 0.3400 - val_loss: 0.8006 - val_soft_acc: 0.3070
Epoch 57/2000
453/453 [==============================] - 0s 258us/step - loss: 0.7452 - soft_acc: 0.3400 - val_loss: 0.7931 - val_soft_acc: 0.3070
Epoch 58/2000
453/453 [==============================] - 0s 273us/step - loss: 0.7440 - soft_acc: 0.3400 - val_loss: 0.7918 - val_soft_acc: 0.3070
Epoch 59/2000
453/453 [==============================] - 0s 256us/step - loss: 0.7435 - soft_acc: 0.3400 - val_loss: 0.7915 - val_soft_acc: 0.3070
Epoch 60/2000
453/453 [==============================] - 0s 267us/step - loss: 0.7421 - soft_acc: 0.3400 - val_loss: 0.7877 - val_soft_acc: 0.3070
Epoch 61/2000
453/453 [==============================] - 0s 272us/step - loss: 0.7432 - soft_acc: 0.3400 - val_loss: 0.7876 - val_soft_acc: 0.3070
Epoch 62/2000
453/453 [==============================] - 0s 272us/step - loss: 0.7406 - soft_acc: 0.3400 - val_loss: 0.7935 - val_so

Epoch 112/2000
453/453 [==============================] - 0s 272us/step - loss: 0.7255 - soft_acc: 0.3400 - val_loss: 0.7698 - val_soft_acc: 0.3070
Epoch 113/2000
453/453 [==============================] - 0s 269us/step - loss: 0.7248 - soft_acc: 0.3400 - val_loss: 0.7668 - val_soft_acc: 0.3070
Epoch 114/2000
453/453 [==============================] - 0s 271us/step - loss: 0.7248 - soft_acc: 0.3400 - val_loss: 0.7694 - val_soft_acc: 0.3070
Epoch 115/2000
453/453 [==============================] - 0s 266us/step - loss: 0.7263 - soft_acc: 0.3400 - val_loss: 0.7712 - val_soft_acc: 0.3070
Epoch 116/2000
453/453 [==============================] - 0s 270us/step - loss: 0.7240 - soft_acc: 0.3400 - val_loss: 0.7696 - val_soft_acc: 0.3070
Epoch 117/2000
453/453 [==============================] - 0s 271us/step - loss: 0.7240 - soft_acc: 0.3400 - val_loss: 0.7685 - val_soft_acc: 0.3070
Epoch 118/2000
453/453 [==============================] - 0s 268us/step - loss: 0.7233 - soft_acc: 0.3400 - val_

453/453 [==============================] - 0s 258us/step - loss: 0.7126 - soft_acc: 0.3400 - val_loss: 0.7548 - val_soft_acc: 0.3070
Epoch 168/2000
453/453 [==============================] - 0s 269us/step - loss: 0.7130 - soft_acc: 0.3400 - val_loss: 0.7523 - val_soft_acc: 0.3070
Epoch 169/2000
453/453 [==============================] - 0s 276us/step - loss: 0.7129 - soft_acc: 0.3400 - val_loss: 0.7542 - val_soft_acc: 0.3070
Epoch 170/2000
453/453 [==============================] - 0s 276us/step - loss: 0.7124 - soft_acc: 0.3400 - val_loss: 0.7508 - val_soft_acc: 0.3070
Epoch 171/2000
453/453 [==============================] - 0s 269us/step - loss: 0.7110 - soft_acc: 0.3400 - val_loss: 0.7499 - val_soft_acc: 0.3070
Epoch 172/2000
453/453 [==============================] - 0s 274us/step - loss: 0.7105 - soft_acc: 0.3400 - val_loss: 0.7535 - val_soft_acc: 0.3070
Epoch 173/2000
453/453 [==============================] - 0s 302us/step - loss: 0.7113 - soft_acc: 0.3400 - val_loss: 0.7501 - 

453/453 [==============================] - 0s 270us/step - loss: 0.6731 - soft_acc: 0.3400 - val_loss: 0.6878 - val_soft_acc: 0.3070
Epoch 223/2000
453/453 [==============================] - 0s 276us/step - loss: 0.6711 - soft_acc: 0.3400 - val_loss: 0.6966 - val_soft_acc: 0.3070
Epoch 224/2000
453/453 [==============================] - 0s 280us/step - loss: 0.6732 - soft_acc: 0.3400 - val_loss: 0.6963 - val_soft_acc: 0.3070
Epoch 225/2000
453/453 [==============================] - 0s 281us/step - loss: 0.6718 - soft_acc: 0.3400 - val_loss: 0.6899 - val_soft_acc: 0.3070
Epoch 226/2000
453/453 [==============================] - 0s 277us/step - loss: 0.6707 - soft_acc: 0.3400 - val_loss: 0.6851 - val_soft_acc: 0.3070
Epoch 227/2000
453/453 [==============================] - 0s 278us/step - loss: 0.6692 - soft_acc: 0.3400 - val_loss: 0.6904 - val_soft_acc: 0.3070
Epoch 228/2000
453/453 [==============================] - 0s 263us/step - loss: 0.6689 - soft_acc: 0.3400 - val_loss: 0.6862 - 

...

453/453 [==============================] - 0s 298us/step - loss: 0.3912 - soft_acc: 0.7196 - val_loss: 0.5408 - val_soft_acc: 0.5263
Epoch 1213/2000
453/453 [==============================] - 0s 297us/step - loss: 0.3908 - soft_acc: 0.7241 - val_loss: 0.5349 - val_soft_acc: 0.5175
Epoch 1214/2000
453/453 [==============================] - 0s 276us/step - loss: 0.3912 - soft_acc: 0.7219 - val_loss: 0.5559 - val_soft_acc: 0.5439
Epoch 1215/2000
453/453 [==============================] - 0s 286us/step - loss: 0.3928 - soft_acc: 0.7329 - val_loss: 0.5549 - val_soft_acc: 0.5526
Epoch 1216/2000
453/453 [==============================] - 0s 334us/step - loss: 0.3894 - soft_acc: 0.7152 - val_loss: 0.5367 - val_soft_acc: 0.5263
Epoch 1217/2000
453/453 [==============================] - 0s 274us/step - loss: 0.3893 - soft_acc: 0.7108 - val_loss: 0.5339 - val_soft_acc: 0.5263
Epoch 1218/2000
453/453 [==============================] - 0s 305us/step - loss: 0.3924 - soft_acc: 0.7152 - val_loss: 0.5

453/453 [==============================] - 0s 270us/step - loss: 0.3881 - soft_acc: 0.7196 - val_loss: 0.5332 - val_soft_acc: 0.5263
Epoch 1268/2000
453/453 [==============================] - 0s 278us/step - loss: 0.3846 - soft_acc: 0.7241 - val_loss: 0.5521 - val_soft_acc: 0.5088
Epoch 1269/2000
453/453 [==============================] - 0s 282us/step - loss: 0.3900 - soft_acc: 0.7174 - val_loss: 0.5315 - val_soft_acc: 0.5439
Epoch 1270/2000
453/453 [==============================] - 0s 277us/step - loss: 0.3866 - soft_acc: 0.7329 - val_loss: 0.5344 - val_soft_acc: 0.5439
Epoch 1271/2000
453/453 [==============================] - 0s 281us/step - loss: 0.3815 - soft_acc: 0.7219 - val_loss: 0.5553 - val_soft_acc: 0.5088
Epoch 1272/2000
453/453 [==============================] - 0s 272us/step - loss: 0.3974 - soft_acc: 0.7020 - val_loss: 0.5383 - val_soft_acc: 0.5175
Epoch 1273/2000
453/453 [==============================] - 0s 288us/step - loss: 0.3848 - soft_acc: 0.7130 - val_loss: 0.5

453/453 [==============================] - 0s 269us/step - loss: 0.3842 - soft_acc: 0.7285 - val_loss: 0.5377 - val_soft_acc: 0.5614
Epoch 1323/2000
453/453 [==============================] - 0s 281us/step - loss: 0.3795 - soft_acc: 0.7196 - val_loss: 0.5870 - val_soft_acc: 0.5614
Epoch 1324/2000
453/453 [==============================] - 0s 276us/step - loss: 0.3845 - soft_acc: 0.7174 - val_loss: 0.5392 - val_soft_acc: 0.5614
Epoch 1325/2000
453/453 [==============================] - 0s 273us/step - loss: 0.3812 - soft_acc: 0.7020 - val_loss: 0.5419 - val_soft_acc: 0.5263
Epoch 1326/2000
453/453 [==============================] - 0s 270us/step - loss: 0.3830 - soft_acc: 0.7174 - val_loss: 0.5377 - val_soft_acc: 0.5263
Epoch 1327/2000
453/453 [==============================] - 0s 303us/step - loss: 0.3769 - soft_acc: 0.7285 - val_loss: 0.6347 - val_soft_acc: 0.5614
Epoch 1328/2000
453/453 [==============================] - 0s 274us/step - loss: 0.3936 - soft_acc: 0.7241 - val_loss: 0.5

453/453 [==============================] - 0s 268us/step - loss: 0.3792 - soft_acc: 0.7285 - val_loss: 0.5381 - val_soft_acc: 0.5000
Epoch 1378/2000
453/453 [==============================] - 0s 279us/step - loss: 0.3824 - soft_acc: 0.7285 - val_loss: 0.5460 - val_soft_acc: 0.5526
Epoch 1379/2000
453/453 [==============================] - 0s 272us/step - loss: 0.3802 - soft_acc: 0.7351 - val_loss: 0.5589 - val_soft_acc: 0.5614
Epoch 1380/2000
453/453 [==============================] - 0s 272us/step - loss: 0.3770 - soft_acc: 0.7373 - val_loss: 0.5376 - val_soft_acc: 0.5351
Epoch 1381/2000
453/453 [==============================] - 0s 271us/step - loss: 0.3798 - soft_acc: 0.7395 - val_loss: 0.5457 - val_soft_acc: 0.5789
Epoch 1382/2000
453/453 [==============================] - 0s 273us/step - loss: 0.3772 - soft_acc: 0.7351 - val_loss: 0.5334 - val_soft_acc: 0.5351
Epoch 1383/2000
453/453 [==============================] - 0s 271us/step - loss: 0.3754 - soft_acc: 0.7395 - val_loss: 0.5

453/453 [==============================] - 0s 263us/step - loss: 0.3753 - soft_acc: 0.7373 - val_loss: 0.5546 - val_soft_acc: 0.5526
Epoch 1433/2000
453/453 [==============================] - 0s 268us/step - loss: 0.3755 - soft_acc: 0.7439 - val_loss: 0.5582 - val_soft_acc: 0.5439
Epoch 1434/2000
453/453 [==============================] - 0s 273us/step - loss: 0.3725 - soft_acc: 0.7528 - val_loss: 0.5458 - val_soft_acc: 0.5175
Epoch 1435/2000
453/453 [==============================] - 0s 272us/step - loss: 0.3745 - soft_acc: 0.7351 - val_loss: 0.5456 - val_soft_acc: 0.5614
Epoch 1436/2000
453/453 [==============================] - 0s 282us/step - loss: 0.3731 - soft_acc: 0.7461 - val_loss: 0.5279 - val_soft_acc: 0.5263
Epoch 1437/2000
453/453 [==============================] - 0s 273us/step - loss: 0.3704 - soft_acc: 0.7263 - val_loss: 0.5547 - val_soft_acc: 0.5263
Epoch 1438/2000
453/453 [==============================] - 0s 273us/step - loss: 0.3782 - soft_acc: 0.7307 - val_loss: 0.5

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

soft_acc = history.history['soft_acc']
soft_val_acc = history.history['val_soft_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(soft_acc) + 1)
plt.figure(figsize=(30,5))
plt.plot(epochs, soft_acc, 'bo', label='Soft Training acc')
plt.plot(epochs, soft_val_acc, 'r', label='Soft Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure(figsize=(30,5))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()




### 10 Fold Validation

In [104]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvs_scores = []
for train, test in kfold.split(data, tags):
    model = Sequential()
    model.add(Conv1D(8, 10, activation='relu',kernel_regularizer=l1(0.038), input_shape=(1000,45)))
#     model.add(Conv1D(128, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.01), input_shape=(1000,45)))
#     model.add(MaxPooling1D(5))
#     model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
#     model.add(MaxPooling1D(5))
#     model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.0003, l2=0.0003)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1))
    def soft_acc(y_true, y_pred):
        from tensorflow.python.keras import backend as K
        return K.mean(K.equal(K.round(y_true), K.round(y_pred)))
    model.compile(optimizer=RMSprop(),loss='mse',metrics=[soft_acc])
    model.fit(data[train], tags[train], epochs=1000, verbose=0)
    scores = model.evaluate(data[test], tags[test], verbose=0)
    print(scores[1]*100)
    cvs_scores.append(scores[1]*100)
print("Accuracy:", np.mean(cvs_scores),"std:", np.std(cvs_scores))

31.578947995838362
35.087719062964126
36.842105315442666
49.12280712211341
52.6315792087923
54.38596501685026
64.91228122460214
45.61403498314975
49.122806180987446
42.59259237183465
Accuracy: 46.18908384825751 std: 9.559257271784583


### Evaluation on old dataset

In [108]:
model.evaluate(old_data, old_tags)


183/183 [==============================] - 0s 159us/step


[0.7454337044491794, 0.44262295092145604]